<a href="https://colab.research.google.com/github/hananather/DCP/blob/main/Python_Programming_and_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classes and Interfances
A **class** is a blueprint for creating objects in Python. An **object** is a collection of data (variables) and methods (functions) that act on that data. 

In Python, **interfaces** are not explicitly built into the language but are implemented using a concept called *duck typing*, which means an object's suitability is determined by the pressence of certain methods and properties, rather than the type of the object itself.

Python has a module named `abc` which provides the infrastructure for defining abstract base classes (ABCs). ABCs are a form of interface checking more strict than conventional duck typing. 

## Distribution Interface



Ho can we reresent a distribution in code?
What can we do with this distribution?
How can we write code to get the expected value of a distribution?


Sampling is the least common denominator. We can sample from distributions where we don't know enough to do anything else, and we can sample distributions where we know the exact form of the distribution.

In [ ]:
from abc import ABC, abstractmethod
import random

class Distribution(ABC):
  @abstractmethod
  def sample(self):
    pass

We have made distribution an abstract base class (ABC).
This class  defines an **interface**: a definition of what we require for something to quantify as distribuiton. Any kind of distribution we implement in the future will be able to, at minimum, generate sample.

In [ ]:
class Die(Distribution):
  def __init__(self, sides):
    self.sides = sides

  def sample(self):
    return random.randint(1, self.sides)
  
  def __repr__(self):
    return f"Die(sides= {self.sides})"

  def _eq_(self, other):
    if isinstance(other, Die):
      return self.sides == other.sides
    return False

    
six_sided = Die(6)
six_sided.sample()

4

In this example, **`Distribution`** is an abstract base class that declares the interface for `Die`. Any class that wants to implement `Distribution` must implement `sample`.

# Decorators
**Decorators** in Python are a powerful and useful feature that allows you to modify or extend the behaviour of functions or methods without changing their code. **They are essentially higher-order functions that take a function as input and return a new function with added functionaility**.

### Function as objects 
In Pythonm functions are first-class objects. This means that you can treat them like objects, such as integers, stringm or list. You can pass them as arguments to other functions, assign them to variables, and store them in data structures.

### Higher-order functions 
A higher-order function is a function that takes one or more functions, and/or returns a function as its result. A decorator is a higher-order function that takes a function as input and returns a new function with added functionality. You can think of decorators as wrappers that modify the behaviour of the input function. 

### Using the @ syntax
Python provides a more convenient way to apply decorators using the '@' symbol, follwed by decorator's name. This makes the code more readable and easier to understand. 

## Step-by-step: `*args` and `**kwargs`
`*args` and `**kwargs` are special syntax in Python for passing a variable number of arguments to a function. They allow us to write more flexible functions that can accept any number of arguments

### understanding `*args`:
`*args` is used to pass variable number of non-keyword (positional) arguments to a function. It allows you to pass any number of positional arguments to the function, which will be recived as a tuple.

### Dataclasses
Python decorators are modifiers that can be applied to class, function and method definitions. A decorator is written above the definition that it applies to, starting with an @ symbol.

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Die(Distribution):
  sides: int
  
  def sample(self) -> int:
    return random.randint(1, self.sides)

### Immutability
An object we cannot change is called immutable. Instead of changing the object we cab return a fresh copy with the attribute changed; `dataclasses` providdes a `replace` function that makes this easy. Returning a fresh copy of data rather than modifying in place is a common pattern in Python Libraries. 

In [ ]:
d = Die(6)
d.sample()
d.sides = 6

FrozenInstanceError: ignored

Different kinds of distribution-- different implementations of the `Distribution` interface -- will return different types of samples. To deal with this we need **type variables**: variables that stand in for some type that can be different in different variables. These type variables are known as *generics* because they let us write classses that generically work for any type. To add annotations to the abstract `Distribution` class, we need to define a type variable for the outcomes of the distribution, then tell python that `Distribution` is "generic" in that type.

In [ ]:
from typing import Generic, TypeVar, Sequence

# A type variable named "A"
A = TypeVar("A")

# Distribution is "generic in A"
class Distribution(ABC, Generic[A]):

  # Sampling must produce a value of type A
  @abstractmethod 
  def sample(self) -> A:
    pass
  
  def sample_n(self, n:int) -> Sequence[A]:
    return [self.sample() for _ in range(n)]

In this code, we have defined a type variable A and specified that `Distribution` uses `A` by interithing from `Generic[A]`. We can now write type annotations for distribution with specific types of outcomes: for example, `Die` would be an instance of `Distribution[int]` since the outcome of a die roll is always an `int`.

In [ ]:
import statistics

@dataclass(frozen=True)
class Die(Distribution[int]):
  sides: int
  
  def sample(self) -> int:
    return random.randint(1, self.sides)
  
  def exptected_value(d: Distribution[float], n:int = 100) -> float:
    return statistics.mean(d.sample() for _ in range(n))


In [ ]:
def exptected_value(d: Distribution[float], n:int = 100) -> float:
  return statistics.mean(d.sample() for _ in range(n))
exptected_value(Die(6))

3.46

In [ ]:
from __future__ import annotations

from  abc import ABC, abstractmethod
from collections import Counter, defaultdict
from dataclasses import dataclass
import numpy as np
import random
from typing import (Callable, Dict, Generic, Iterator, Iterable,
                    Mapping, Optional, Sequence, Tuple, TypeVar)

Typing is a python module that provides runtime support for type hints. 

# Interfaces 
Why are interfaces useful?

1. Promote a standard protocol: Interfaces can help you define a standard protocol that must be followed by all classes that implement the interface. This is useful when you want different classes to have the same methods, so they can be used interchangebly.

2. Enfornce Contracts: With the `abc` module, Python can enforce that child classes implement particular methods from the parents class. This is ufeful in large codebases where you want ot ensure certian method are alwayas implement in subclasses.

## First Class Functions


In [ ]:
for _ in range(10):
  do_something()

NameError: ignored

Instead of writing a loop each time, we could factor the logic into a function that takes in $n$ and `do_something` as arguments: 

In [ ]:
def repeat(action: Callable, n:int):
  for _ in range(n):
    action()

repeat(do_something)

NameError: ignored

In [ ]:
def expected_value(d: Distribution[float], n: float) -> float:
  return statistics.mean(d.sample() for _ in range(n))

NameError: ignored

In [ ]:
def expected_value(
    d: Distribution[A],
    f: Callable[[A], float],
    n: int
) -> float:
  return statistics.mean(f(d.sample()) for _ in range(n))

NameError: ignored

## Iterative Algorithms

One common scenario in Reinforcement learning -- and other areas in numerical programming is algorithms that *iteratively converge* to the correct result. We can run the algorithm repeatedly to get more and more accurate results, but the improvments with each iteration get progressively smaller. For example we can approximate the square root of $a$ by starting with some inital guess $x_0$ and repeatedly calculating $x_{n+1}$

In [ ]:
def sqrt(a: float, threshold: float) -> float:
  x = a / 2 # initial guess
  x_n = a 
  while abs(x_n - x) > threshold:
    x_n = x
    x = (x + (a / x)) / 2
    print(x)
  return x_n 

In [ ]:
sqrt(100, 0.001)

26.0
14.923076923076923
10.812053925455988
10.030495203889796
10.000046356507898
10.000000000107445


10.000046356507898

In [ ]:
def sqrt(a: float, threshold: float) -> float:
    x = a / 2  # initial guess
    x_n = a
    while abs(x_n - x) > threshold:
        x_n = x
        x = (x + (a / x)) / 2
        print(x, x_n)
    return x_n